In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 4"

In [2]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
import optuna
import gc
xgb.__version__

/home/tunguz/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'1.6.0rc1'

In [3]:
train_x = dd.read_csv('../../TPS_2021/input/tabular-playground-series-oct-2021/xgtrain.csv')
test_x = dd.read_csv('../../TPS_2021/input/tabular-playground-series-oct-2021/xgval.csv')

In [4]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [5]:
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES=["GPU-a19c00c3-2832-fe38-1c43-c18db3e909da",
                                                 "GPU-58b97c92-e879-49d3-85b5-1d9615f10873",
                                                 "GPU-d21cfed4-2e1a-f313-839c-ea008aca027a",
                                                 "GPU-e3b349d7-ac6c-77ab-3564-ed9d05d50bac"])
client = Client(cluster)

2022-05-06 15:31:40,055 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-06 15:31:40,075 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-06 15:31:40,088 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-06 15:31:40,112 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [6]:
train_x.head()

,f0,f1,f10,f100,f101,f102,f103,f104,f105,f106,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
0,-0.824626,-0.422467,0.554079,-0.856495,0.317947,-0.256859,187.144413,0.869936,-0.531424,-1.019996,...,0.352066,0.037746,0.320359,0.795707,-1.038934,-0.269760,35.650042,-0.697299,1.931045,-0.305331
1,-1.021557,-0.815959,-0.002685,0.022667,0.025057,-0.424646,0.506253,-0.078752,-0.018520,-0.582415,...,0.535939,0.112756,0.046762,0.161233,0.184715,-0.087651,120.244783,0.402223,-0.013730,0.513328
2,0.062020,-0.837565,1.121388,1.519331,0.583978,58.726697,-0.375958,-0.095803,0.522253,-0.502932,...,0.925234,0.276785,-0.599029,0.111792,0.395261,0.166551,-0.775042,0.283871,0.899296,-0.153473
3,0.514858,0.488789,-0.473529,0.352311,-0.917443,-0.783741,-0.871722,-0.082538,0.489024,-0.012730,...,-0.277384,-1.389723,1.848743,1.228182,74.222184,-0.444784,-0.360392,-0.765911,-0.043319,34.474152
4,-0.875484,1.092204,-0.547237,1.015510,2.688259,-0.873262,0.550222,2.858554,-0.496966,-0.920987,...,0.175718,-2.240798,0.863892,0.613538,-0.279954,54.065426,24.236644,0.426447,0.991493,-0.895396


In [7]:
dtrain = xgb.dask.DaskDMatrix(client, train_x, train_y)
dtest = xgb.dask.DaskDMatrix(client, test_x, test_y)
num_round = 1000

In [8]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']), 
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()

    roc = roc_auc_score(test_y, predictions)
    
    return roc

In [9]:
study = optuna.create_study(direction='maximize')

[I 2022-05-06 15:32:49,969] A new study created in memory with name: no-name-e4f88b99-f88c-4d32-9f30-eb5b20a4cf5b


In [11]:
%%time
study.optimize(objective, n_trials=2)

[15:33:16] task [xgboost.dask]:tcp://127.0.0.1:38933 got new rank 0
[15:33:16] task [xgboost.dask]:tcp://127.0.0.1:40841 got new rank 1
[15:33:16] task [xgboost.dask]:tcp://127.0.0.1:44275 got new rank 2
[15:33:16] task [xgboost.dask]:tcp://127.0.0.1:39297 got new rank 3
[I 2022-05-06 15:35:35,090] Trial 0 finished with value: 0.8557663987624522 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 4.69585995721641, 'alpha': 2.984940515866779, 'colsample_bytree': 0.4270608818086083, 'subsample': 0.7416502715788436, 'learning_rate': 0.01280395218181446, 'max_depth': 17, 'min_child_weight': 173, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.8557663987624522.
[15:35:35] task [xgboost.dask]:tcp://127.0.0.1:38933 got new rank 0
[15:35:35] task [xgboost.dask]:tcp://127.0.0.1:44275 got new rank 1
[15:35:35] task [xgboost.dask]:tcp://127.0.0.1:39297 got new rank 2
[15:35:35] task [xgboost.dask]:tcp://127.0.0.1:40841 got new rank 3
[I 2022-05-06 15:36:

CPU times: user 16.2 s, sys: 5.89 s, total: 22.1 s
Wall time: 2min 48s


In [12]:
%%time
study.optimize(objective, n_trials=5)

[15:36:04] task [xgboost.dask]:tcp://127.0.0.1:38933 got new rank 0
[15:36:04] task [xgboost.dask]:tcp://127.0.0.1:44275 got new rank 1
[15:36:04] task [xgboost.dask]:tcp://127.0.0.1:40841 got new rank 2
[15:36:04] task [xgboost.dask]:tcp://127.0.0.1:39297 got new rank 3
[I 2022-05-06 15:36:34,394] Trial 2 finished with value: 0.8435712365063539 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.0023695358718639785, 'alpha': 0.16697050273216607, 'colsample_bytree': 0.3823118666536898, 'subsample': 0.8606198401945112, 'learning_rate': 0.001056716413000844, 'max_depth': 7, 'min_child_weight': 204, 'eval_metric': 'logloss'}. Best is trial 1 with value: 0.8562849312212153.
[15:36:34] task [xgboost.dask]:tcp://127.0.0.1:38933 got new rank 0
[15:36:34] task [xgboost.dask]:tcp://127.0.0.1:39297 got new rank 1
[15:36:34] task [xgboost.dask]:tcp://127.0.0.1:44275 got new rank 2
[15:36:34] task [xgboost.dask]:tcp://127.0.0.1:40841 got new rank 3
[I 2022-05-06

CPU times: user 9.5 s, sys: 1.27 s, total: 10.8 s
Wall time: 2min 4s


/home/tunguz/.local/lib/python3.8/site-packages/dask_cuda/initialize.py:57: UserWarning: Worker with process ID 1150465 should have a CUDA context assigned to device GPU-a19c00c3-2832-fe38-1c43-c18db3e909da, but instead the CUDA context is on device 0. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warnings.warn(
/home/tunguz/.local/lib/python3.8/site-packages/dask_cuda/initialize.py:57: UserWarning: Worker with process ID 1150462 should have a CUDA context assigned to device GPU-58b97c92-e879-49d3-85b5-1d9615f10873, but instead the CUDA context is on device 1. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warning